In [ ]:
from climpyrical.gridding import find_nearest_index, find_element_wise_nearest_pos, scale_model_obs
from climpyrical.mask import gen_raster_mask_from_vector, gen_upper_archipelago_mask, stratify_coords
import climpyrical.spytialProcess as sp
from climpyrical.data import read_data, interpolate_dataset, gen_dataset
from climpyrical.rkrig import krigit_north, rkrig_r
from climpyrical.cmd.find_matched_model_vals import add_model_values

from pkg_resources import resource_filename

from rpy2.robjects.packages import importr
from sklearn.neighbors import NearestNeighbors
import warnings

import pandas as pd
import numpy as np
import config

import matplotlib.pyplot as plt

importr("fields")


%load_ext autoreload
%autoreload 2

In [ ]:
# parameter cell
# these currently have default values but are configurable when 
# executing this notebook with papermill

# station_dv = "RL50 (kPa)"
# station_input_path = resource_filename("climpyrical", "data/station_inputs/Interim_snow_rain_load_LR_composite_stations_tbd_v4.csv")
# name = "RL50"
# processed_model_output_path = resource_filename("climpyrical", "data/results/intermediate/preprocessed_models/RL50_preprocessed.nc")
# output_reconstruction_path = resource_filename("climpyrical", f"data/results/datasets/RL50_reconstructed.nc")
# df_path_write = resource_filename('climpyrical', f'data/results/intermediate/preprocessed_stations/RL50_processed_stations.csv')

In [ ]:
dsold_max = 28.15999984741211

mask_path = resource_filename(
    'climpyrical',
    'data/masks/canada_mask_rp.nc'
)

north_mask_path = resource_filename(
    'climpyrical',
    'data/masks/canada_mask_north_rp.nc'
)

ds = read_data(processed_model_output_path)
(dv, ) = ds.data_vars
mask = read_data(mask_path)['mask'].values
northern_mask = read_data(north_mask_path)['mask'].values

rlon, rlat = np.meshgrid(ds.rlon, ds.rlat)

In [ ]:
df = pd.read_csv(df_path_write, index_col=False)
df.head(3)

separate stations into beyond and within the model domain

In [ ]:
df_north = df[df.rlat > dsold_max].copy()
df_south = df[df.rlat <= dsold_max].copy()

north_index = df_north.index.values
south_index = df_south.index.values

In [ ]:
X_distances = np.stack([np.deg2rad(df_south.lat.values), np.deg2rad(df_south.lon.values)])
nbrs = NearestNeighbors(n_neighbors=30, metric="haversine").fit(
    X_distances.T
)

# Order independent window checkers
# only uses windows that are not-identical

dist, ind = nbrs.kneighbors(X_distances.T)
good_i = []
list_of_sets = []
count = 0 
for i in range(df_south.shape[0]):
    list_of_sets.append(df_south[["lon", "lat", station_dv]].iloc[ind[i]].values)
    if i+1-count == np.unique(list_of_sets, axis=0).shape[0]:
        good_i.append(i)
    else:
        warning.warn("There are identical windows!")
        count += 1 

df_south = df_south.iloc[good_i]

# Krig the station only values in the north

In [ ]:
UAA_station_mean = np.nanmean(df[station_dv][df.rlat > dsold_max-1])

In [ ]:
# choose starting value
# model_vals = ds[dv].values[df.ilocy.values, df.ilocx.values]
model_vals = df.model_values
station_vals = df[station_dv]

ratio, best_tol = scale_model_obs(df.model_values, station_vals)

# apply correction
model_vals_corrected = (model_vals/best_tol)
mean_corrected = (ds[dv].values/best_tol)

print("These should be similar: ", best_tol, np.sum(df.model_values)/np.sum(station_vals))

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
ratio_field = rkrig_r(df_south, 30, ds, station_dv)
ratio_field[~mask] = np.nan

In [ ]:
752, 407
ratio_field[752, 407]

In [ ]:
selection = ~np.isnan(ds[dv].where((ds.lat > 72.) & (ds.lat < 73) & (ds.lon - 360 < -75) & (ds.lon - 360 > -127)))

nanmask = ~np.isnan(ratio_field)

points = np.stack([rlon[nanmask], rlat[nanmask]]).T
target_points = np.stack([rlon[nanmask^mask], rlat[nanmask^mask]]).T

# swap the filling steps if either of these DVs
if station_dv == "TJul2.5 (degC" or station_dv == "TwJul2.5 (degC)":
    reconstructed_field = ratio_field*mean_corrected.copy()
    target_values = reconstructed_field[nanmask]
    reconstructed_field[nanmask^mask] = interpolate_dataset(points, target_values, target_points, 'nearest')
else:
    target_values = ratio_field[nanmask]
    ratio_field[nanmask^mask] = interpolate_dataset(points, target_values, target_points, 'nearest')
    reconstructed_field = ratio_field*mean_corrected.copy()

reconstructed_field_strip_mean = np.nanmean(reconstructed_field[selection])
combined_ratio_station_mean = np.mean([reconstructed_field_strip_mean, UAA_station_mean])
reconstructed_field[northern_mask] = combined_ratio_station_mean

In [ ]:
print("Recon field", reconstructed_field[752, 407], 
      "\nMean correct", mean_corrected[752, 407], 
      "\nRatio", ratio_field[752, 407], 
      "\nMean non-corrected", ds[dv].values[752, 407], 
      "\nGlobal bias shift", best_tol)

In [ ]:
import seaborn as sns

sns.set_theme(style="whitegrid")

fig, ax = plt.subplots(figsize=(8, 4.5))

violindata = np.concatenate([df.ratio, ratio_field[df.irlat, df.irlon]])

b_str = ["B" for x in df.ratio.values]
bp_str = ["B'" for x in ratio_field[df.irlat, df.irlon]]

vstrings = np.concatenate([b_str, bp_str])

vdf = pd.DataFrame({"Ratios at Station Grid Cells (i)": violindata, '': vstrings})
# plt.tight_layout()

ax.set_title(f"{name} Distributions of Results", fontsize=20)
sns.violinplot(ax=ax, x="Ratios at Station Grid Cells (i)", y='', data=vdf, palette=sns.color_palette('pastel'))

In [ ]:
print(
    "Northern fill value:"
    "\n"
    "Reconstruction", reconstructed_field_strip_mean,
    "\n"
    "UAA_station_mean", UAA_station_mean,
    "\n"
    "Combined", combined_ratio_station_mean
)

In [ ]:
if ds[dv].attrs["units"]=="K":
    print("Convert back to degC")
    K = 273.15 # K
    reconstructed_field -= K
    df[station_dv] -= K
    ds[dv].attrs["units"] = "degC"

In [ ]:
ds_recon = gen_dataset(dv, reconstructed_field, ds.rlat, ds.rlon, ds.lat, ds.lon, unit=ds[dv].attrs["units"])

In [ ]:
if ds.attrs:
    all_keys = set(ds_recon.variables).union(set(ds_recon.dims))
    for key in all_keys:
        ds_recon[key].attrs = ds[key].attrs
    attr_dict = ds.attrs
    attr_dict["Climpyrical"] = (
        "CanRCM4 Reconstruction contains"
        "hybrid station and model data using"
        "Climpyrical (https://github.com/pacificclimate/climpyrical)"
    )

    ds_recon.attrs = attr_dict
else:
    raise warnings.warn("No attributes detected in dataset file")

In [ ]:
ds_recon.to_netcdf(output_reconstruction_path, 'w')

In [ ]:
plt.figure(figsize=(20, 20))
plt.title(f"{name} reconstruction", fontsize=25)
plt.imshow(ds_recon[dv], origin='lower')